g

In [1]:
# =====================================================
# COZY CRAFT COLLECTIVE
# ORGANIZATIONAL ANALYTICS – FINAL PRODUCTION BUILD
# =====================================================

import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)

print("="*70)
print("SECTION 1 – DATA INGESTION")
print("="*70)

# -----------------------------
# LOAD RAW EXPORTS
# -----------------------------

orders = pd.read_csv(r"C:\Users\kevin\Downloads\Cross-Event_(6_Events)_Orders_2907567112361_20260209_221944_090.csv")
attendees = pd.read_csv(r"C:\Users\kevin\Downloads\Cross-Event_(6_Events)_Attendees_2907567112361_20260211_060613_498.csv")
checkins = pd.read_csv(r"C:\Users\kevin\Downloads\Cross-Event_(6_Events)_Check-ins_2907567112361_20260209_230334_727.csv")

# -----------------------------
# STANDARDIZE DATE FIELDS
# -----------------------------

for df in [orders, attendees, checkins]:
    df["Event start date"] = pd.to_datetime(df["Event start date"])

print("\nROW COUNTS:")
print("Orders:", len(orders))
print("Attendees:", len(attendees))
print("Checkins:", len(checkins))


SECTION 1 – DATA INGESTION

ROW COUNTS:
Orders: 340
Attendees: 393
Checkins: 392


In [2]:
print("\n" + "="*70)
print("SECTION 2 – CLEAN ORDERS")
print("="*70)

orders_clean = orders.copy()

orders_clean["Year"] = orders_clean["Event start date"].dt.year

event_dim = (
    orders_clean
    .groupby(["Event ID", "Event name", "Event start date", "Year"], as_index=False)
    .agg({
        "Ticket quantity": "sum"
    })
    .rename(columns={"Ticket quantity": "Seats Reserved"})
)

print("\nEVENT DIMENSION PREVIEW:")
print(event_dim.sort_values("Event start date"))



SECTION 2 – CLEAN ORDERS

EVENT DIMENSION PREVIEW:
       Event ID                               Event name Event start date  \
1  1.767767e+12                        Cozy Craft Meetup       2025-10-12   
0  1.712496e+12               November Cozy Craft Meetup       2025-11-09   
2  1.966230e+12               December Cozy Craft Meetup       2025-12-07   
3  1.977406e+12             January Cozy Craft Meetup #1       2026-01-04   
4  1.978879e+12  January Cozy Craft Meetup #2 - Lakewood       2026-01-18   
5  1.980839e+12           February BHM Cozy Craft Meetup       2026-02-08   

     Year  Seats Reserved  
1  2025.0              30  
0  2025.0              55  
2  2025.0              57  
3  2026.0              82  
4  2026.0              76  
5  2026.0              92  


In [5]:
print("\n" + "="*70)
print("SECTION 4 – MERGE VERIFIED COUNTS")
print("="*70)

verified_counts = (
    verified.groupby("Event name")
    .size()
    .reset_index(name="Verified Check-Ins")
)

event_dim = event_dim.merge(
    verified_counts,
    on="Event name",
    how="left"
)

event_dim["Verified Check-Ins"] = event_dim["Verified Check-Ins"].fillna(0)

# Attendance Rate only for 2026
event_dim["Attendance Rate"] = np.where(
    event_dim["Year"] == 2026,
    event_dim["Verified Check-Ins"] / event_dim["Seats Reserved"],
    np.nan
)

print(event_dim.sort_values("Event start date"))



SECTION 4 – MERGE VERIFIED COUNTS
       Event ID                               Event name Event start date  \
1  1.767767e+12                        Cozy Craft Meetup       2025-10-12   
0  1.712496e+12               November Cozy Craft Meetup       2025-11-09   
2  1.966230e+12               December Cozy Craft Meetup       2025-12-07   
3  1.977406e+12             January Cozy Craft Meetup #1       2026-01-04   
4  1.978879e+12  January Cozy Craft Meetup #2 - Lakewood       2026-01-18   
5  1.980839e+12           February BHM Cozy Craft Meetup       2026-02-08   

     Year  Seats Reserved  Verified Check-Ins  Attendance Rate  
1  2025.0              30                 0.0              NaN  
0  2025.0              55                15.0              NaN  
2  2025.0              57                 0.0              NaN  
3  2026.0              82                32.0         0.390244  
4  2026.0              76                21.0         0.276316  
5  2026.0              92          

In [13]:
print("\n" + "="*70)
print("SECTION 3 – BUILD VERIFIED FROM RAW CHECKINS")
print("="*70)

# Add Year to checkins
checkins["Year"] = checkins["Event start date"].dt.year

# TRUE verified = seat-level check-ins
verified = checkins[
    checkins["Check-in status"] == "Checked In"
].copy()

print("Total Verified (All Years):", len(verified))

# Quick 2026 sanity check
verified_2026 = verified[verified["Year"] == 2026]
print("Verified 2026 (raw checkins):", len(verified_2026))



SECTION 3 – BUILD VERIFIED FROM RAW CHECKINS
Total Verified (All Years): 131
Verified 2026 (raw checkins): 115


In [15]:
print(event_dim.columns)


Index(['Event ID', 'Event name', 'Event start date', 'Year', 'Seats Reserved',
       'Verified Check-Ins_x', 'Attendance Rate', 'Verified Check-Ins_y'],
      dtype='object')


In [16]:
print("\n" + "="*70)
print("SECTION 4 – MERGE VERIFIED COUNTS (RESET SAFE)")
print("="*70)

# Rebuild verified_counts clean
verified_counts = (
    verified
    .groupby("Event ID")
    .size()
    .reset_index(name="Verified Check-Ins")
)

# Drop any previous verified column before merge
event_dim = event_dim.drop(
    columns=[col for col in event_dim.columns if "Verified" in col],
    errors="ignore"
)

# Merge clean
event_dim = event_dim.merge(
    verified_counts,
    on="Event ID",
    how="left"
)

event_dim["Verified Check-Ins"] = event_dim["Verified Check-Ins"].fillna(0)

event_dim["Attendance Rate"] = np.where(
    event_dim["Year"] == 2026,
    event_dim["Verified Check-Ins"] / event_dim["Seats Reserved"],
    np.nan
)

print(event_dim.sort_values("Event start date"))



SECTION 4 – MERGE VERIFIED COUNTS (RESET SAFE)
       Event ID                               Event name Event start date  \
1  1.767767e+12                        Cozy Craft Meetup       2025-10-12   
0  1.712496e+12               November Cozy Craft Meetup       2025-11-09   
2  1.966230e+12               December Cozy Craft Meetup       2025-12-07   
3  1.977406e+12             January Cozy Craft Meetup #1       2026-01-04   
4  1.978879e+12  January Cozy Craft Meetup #2 - Lakewood       2026-01-18   
5  1.980839e+12           February BHM Cozy Craft Meetup       2026-02-08   

     Year  Seats Reserved  Attendance Rate  Verified Check-Ins  
1  2025.0              30              NaN                 0.0  
0  2025.0              55              NaN                16.0  
2  2025.0              57              NaN                 0.0  
3  2026.0              82         0.475610                39.0  
4  2026.0              76         0.355263                27.0  
5  2026.0             

In [17]:
print("\n" + "="*70)
print("EXECUTIVE KPI SUMMARY – 2026 BASELINE")
print("="*70)

kpi_2026 = event_dim[event_dim["Year"] == 2026]

total_seats_2026 = kpi_2026["Seats Reserved"].sum()
verified_total_2026 = kpi_2026["Verified Check-Ins"].sum()
attendance_rate_2026 = verified_total_2026 / total_seats_2026

print(f"Total Seats 2026: {int(total_seats_2026)}")
print(f"Verified Check-ins 2026: {int(verified_total_2026)}")
print(f"Attendance Rate 2026: {round(attendance_rate_2026,4)}")



EXECUTIVE KPI SUMMARY – 2026 BASELINE
Total Seats 2026: 250
Verified Check-ins 2026: 115
Attendance Rate 2026: 0.46


In [18]:
print("\n" + "="*70)
print("SECTION 5 – BUYER CONVERSION (2026)")
print("="*70)

# Build 2026 subsets
orders_2026 = orders_clean[orders_clean["Year"] == 2026].copy()
verified_2026 = verified[verified["Year"] == 2026].copy()

buyers_2026 = set(orders_2026["Buyer email"])
attendees_2026 = set(verified_2026["Attendee email"])

buyer_attended = len(buyers_2026.intersection(attendees_2026))
buyer_conversion_rate = buyer_attended / len(buyers_2026)

print("Total Buyers 2026:", len(buyers_2026))
print("Buyers Who Attended:", buyer_attended)
print("Buyer → Attendee Conversion Rate:", round(buyer_conversion_rate,4))



SECTION 5 – BUYER CONVERSION (2026)
Total Buyers 2026: 168
Buyers Who Attended: 74
Buyer → Attendee Conversion Rate: 0.4405


In [19]:
print("\n" + "="*70)
print("SECTION 6 – LOYALTY SEGMENTATION (2026)")
print("="*70)

loyalty = (
    verified_2026
    .groupby("Attendee email")["Event name"]
    .nunique()
    .reset_index(name="events_attended")
)

def tier(x):
    if x >= 3:
        return "Gold"
    elif x == 2:
        return "Silver"
    else:
        return "Bronze"

loyalty["Tier"] = loyalty["events_attended"].apply(tier)

tier_counts = loyalty["Tier"].value_counts()

print("Loyalty Tier Distribution:")
print(tier_counts)

print("\nCore Members (2+ Events):")
print(len(loyalty[loyalty["events_attended"] >= 2]))



SECTION 6 – LOYALTY SEGMENTATION (2026)
Loyalty Tier Distribution:
Tier
Bronze    60
Silver    12
Gold       2
Name: count, dtype: int64

Core Members (2+ Events):
14


In [20]:
print("\n" + "="*70)
print("SECTION 7 – CAPACITY UTILIZATION MODEL")
print("="*70)

target_in_room = 70

attendance_ratio = verified_total_2026 / total_seats_2026
recommended_tickets = round(target_in_room / attendance_ratio)

print(f"Observed Attendance Rate: {round(attendance_ratio*100,2)}%")
print(f"To reliably get {target_in_room} people in-room:")
print(f"Recommended Ticket Release: {recommended_tickets} seats")



SECTION 7 – CAPACITY UTILIZATION MODEL
Observed Attendance Rate: 46.0%
To reliably get 70 people in-room:
Recommended Ticket Release: 152 seats


In [22]:
# ============================================================
# SECTION 8 – BI READY EXPORTS (FINAL CLEAN VERSION)
# ============================================================

import os

print("\n" + "="*70)
print("SECTION 8 – BI READY EXPORTS (FINAL CLEAN VERSION)")
print("="*70)

# ------------------------------------------------------------
# DEFINE EXPORT PATH (YOUR EXACT FOLDER NAME)
# ------------------------------------------------------------

export_path = r"C:\Users\kevin\OneDrive\Desktop\cozy craft colletive"

# Create folder if it does not exist
os.makedirs(export_path, exist_ok=True)

# ------------------------------------------------------------
# BUILD FACT TABLES
# ------------------------------------------------------------

# FACT 1 – Orders (Seat Sales)
fact_orders = orders_clean[[
    "Order ID",
    "Event ID",
    "Event name",
    "Event start date",
    "Year",
    "Buyer email",
    "Ticket quantity",
    "Purchaser city",
    "Purchaser state",
    "Net sales"
]].copy()

# FACT 2 – Attendance (Seat-Level Verified)
fact_attendance = verified[[
    "Event ID",
    "Event name",
    "Event start date",
    "Year",
    "Attendee email",
    "Check-in status"
]].copy()

# DIMENSION – Event Level
dim_event = event_dim.copy()

# ------------------------------------------------------------
# EXPORT FILES
# ------------------------------------------------------------

fact_orders.to_csv(
    os.path.join(export_path, "cozy_fact_orders.csv"),
    index=False
)

fact_attendance.to_csv(
    os.path.join(export_path, "cozy_fact_attendance.csv"),
    index=False
)

dim_event.to_csv(
    os.path.join(export_path, "cozy_dim_event.csv"),
    index=False
)

print("✅ BI-ready files exported successfully")
print("Export location:", export_path)



SECTION 8 – BI READY EXPORTS (FINAL CLEAN VERSION)
✅ BI-ready files exported successfully
Export location: C:\Users\kevin\OneDrive\Desktop\cozy craft colletive


In [23]:
# ============================================================
# FINAL MASTER EXECUTIVE SUMMARY – LOCKED BASELINE
# ============================================================

print("\n" + "="*75)
print("COZY CRAFT COLLECTIVE – OFFICIAL ANALYTICAL BASELINE (2026)")
print("="*75)

# 2026 slice
kpi_2026 = event_dim[event_dim["Year"] == 2026]

total_seats_2026 = int(kpi_2026["Seats Reserved"].sum())
verified_total_2026 = int(kpi_2026["Verified Check-Ins"].sum())
attendance_rate_2026 = round(verified_total_2026 / total_seats_2026, 4)

unique_buyers_2026 = len(set(orders_2026["Buyer email"]))
unique_attendees_2026 = len(set(verified_2026["Attendee email"]))
buyer_conversion_2026 = round(unique_attendees_2026 / unique_buyers_2026, 4)

# Loyalty
core_members = len(loyalty[loyalty["events_attended"] >= 2])

# Capacity model
target_in_room = 70
recommended_tickets = round(target_in_room / attendance_rate_2026)

print(f"\n--- OPERATIONAL PERFORMANCE (2026) ---")
print(f"Seats Released: {total_seats_2026}")
print(f"Verified Check-Ins: {verified_total_2026}")
print(f"Attendance Rate: {attendance_rate_2026 * 100}%")

print(f"\n--- BUYER BEHAVIOR ---")
print(f"Unique Buyers: {unique_buyers_2026}")
print(f"Unique Verified Individuals: {unique_attendees_2026}")
print(f"Buyer Conversion Rate: {buyer_conversion_2026 * 100}%")

print(f"\n--- COMMUNITY STRUCTURE ---")
print(f"Core Members (2+ Events): {core_members}")

print(f"\n--- CAPACITY FORECAST MODEL ---")
print(f"To reliably host {target_in_room} people in-room:")
print(f"Recommended Ticket Release: {recommended_tickets} seats")

print("\n" + "="*75)
print("BASELINE STATUS: VERIFIED & RECONCILED WITH RAW CHECK-INS")
print("="*75)



COZY CRAFT COLLECTIVE – OFFICIAL ANALYTICAL BASELINE (2026)

--- OPERATIONAL PERFORMANCE (2026) ---
Seats Released: 250
Verified Check-Ins: 115
Attendance Rate: 46.0%

--- BUYER BEHAVIOR ---
Unique Buyers: 168
Unique Verified Individuals: 74
Buyer Conversion Rate: 44.05%

--- COMMUNITY STRUCTURE ---
Core Members (2+ Events): 14

--- CAPACITY FORECAST MODEL ---
To reliably host 70 people in-room:
Recommended Ticket Release: 152 seats

BASELINE STATUS: VERIFIED & RECONCILED WITH RAW CHECK-INS


In [24]:
# ============================================================
# FINAL MASTER ORGANIZATIONAL AUDIT – LOCKED & RECONCILED
# ============================================================

print("\n" + "="*75)
print("COZY CRAFT COLLECTIVE – FULL ORGANIZATIONAL PERFORMANCE AUDIT")
print("="*75)

# ------------------------------
# GLOBAL VALIDATION
# ------------------------------

total_seats_all = int(event_dim["Seats Reserved"].sum())
total_verified_all = int(len(verified))
overall_attendance = round(total_verified_all / total_seats_all, 4)

print("\n--- DATA INTEGRITY CHECK ---")
print(f"Events Analyzed: {len(event_dim)}")
print(f"Total Seats Reserved (All Events): {total_seats_all}")
print(f"Total Verified Check-Ins (All Events): {total_verified_all}")
print(f"Overall Verified Attendance Rate: {overall_attendance * 100}%")

# ------------------------------
# 2026 BASELINE
# ------------------------------

kpi_2026 = event_dim[event_dim["Year"] == 2026]

total_seats_2026 = int(kpi_2026["Seats Reserved"].sum())
verified_total_2026 = int(kpi_2026["Verified Check-Ins"].sum())
attendance_rate_2026 = round(verified_total_2026 / total_seats_2026, 4)

print("\n--- 2026 OPERATIONAL BASELINE ---")
print(f"Seats Reserved (2026): {total_seats_2026}")
print(f"Verified Check-Ins (2026): {verified_total_2026}")
print(f"Verified Attendance Rate: {attendance_rate_2026 * 100}%")

# ------------------------------
# BUYER BEHAVIOR
# ------------------------------

unique_buyers_2026 = len(set(orders_2026["Buyer email"]))
unique_attendees_2026 = len(set(verified_2026["Attendee email"]))
buyer_conversion_2026 = round(unique_attendees_2026 / unique_buyers_2026, 4)

print("\n--- BUYER CONVERSION (2026) ---")
print(f"Buyers: {unique_buyers_2026}")
print(f"Buyers Who Attended: {unique_attendees_2026}")
print(f"Buyer → Attendee Conversion Rate: {buyer_conversion_2026 * 100}%")

# ------------------------------
# COMMUNITY STRUCTURE
# ------------------------------

core_members = len(loyalty[loyalty["events_attended"] >= 2])

print("\n--- COMMUNITY STRUCTURE (2026) ---")
print(f"Unique Verified Individuals: {unique_attendees_2026}")
print(f"Core Members (2+ Events): {core_members}")

# ------------------------------
# CAPACITY FORECAST
# ------------------------------

target_in_room = 70
recommended_tickets = round(target_in_room / attendance_rate_2026)

print("\n--- CAPACITY UTILIZATION MODEL ---")
print(f"Observed Attendance Rate: {attendance_rate_2026 * 100}%")
print(f"To reliably host {target_in_room} people in-room:")
print(f"Recommended Ticket Release: {recommended_tickets} seats")

# ------------------------------
# STRATEGIC INTERPRETATION
# ------------------------------

print("\n" + "="*75)
print("STRATEGIC INTERPRETATION")
print("="*75)

print("""
• 2025 functioned as early-stage capacity build.
• Capacity expanded from 30 → 92 seats (+62 net increase).
• 2026 establishes first reliable operational baseline.
• Attendance stabilized near mid-40% conversion.
• Core repeat members provide structural attendance stability.
• Multi-ticket purchasing confirms social booking behavior.
• Buyer conversion ~44% indicates strong purchase intent alignment.
• Infrastructure now supports predictive capacity modeling.
""")

print("="*75)
print("STATUS: ANALYTICALLY VERIFIED AGAINST RAW CHECK-INS")
print("="*75)



COZY CRAFT COLLECTIVE – FULL ORGANIZATIONAL PERFORMANCE AUDIT

--- DATA INTEGRITY CHECK ---
Events Analyzed: 6
Total Seats Reserved (All Events): 392
Total Verified Check-Ins (All Events): 131
Overall Verified Attendance Rate: 33.42%

--- 2026 OPERATIONAL BASELINE ---
Seats Reserved (2026): 250
Verified Check-Ins (2026): 115
Verified Attendance Rate: 46.0%

--- BUYER CONVERSION (2026) ---
Buyers: 168
Buyers Who Attended: 74
Buyer → Attendee Conversion Rate: 44.05%

--- COMMUNITY STRUCTURE (2026) ---
Unique Verified Individuals: 74
Core Members (2+ Events): 14

--- CAPACITY UTILIZATION MODEL ---
Observed Attendance Rate: 46.0%
To reliably host 70 people in-room:
Recommended Ticket Release: 152 seats

STRATEGIC INTERPRETATION

• 2025 functioned as early-stage capacity build.
• Capacity expanded from 30 → 92 seats (+62 net increase).
• 2026 establishes first reliable operational baseline.
• Attendance stabilized near mid-40% conversion.
• Core repeat members provide structural attendance